##### Copyright 2019 The TensorFlow Authors.

In [4]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 RNN 生成文本

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

本教程演示瞭如何使用基於字符的 RNN 生成文本。您將使用來自 Andrej Karpathy 的 [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) 的莎士比亞作品數據集。給定來自該數據的字符序列（“Shakespear”），訓練一個模型來預測序列中的下一個字符（“e”）。通過重複調用模型可以生成更長的文本序列。
注意：啟用 GPU 加速可以更快地執行此筆記本。在 Colab 中：*運行時 > 更改運行時類型 > 硬件加速器 > GPU*。

本教程包括使用 [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) 和 [eager execution](https://www.tensorflow.org/guide/eager) 實現的可運行代碼。以下是本教程中的模型訓練 30 個 epoch，並以提示“Q”開始時的示例輸出：

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

雖然有些句子是合乎語法的，但大多數都沒有意義。該模型尚未學習單詞的含義，但考慮：

*該模型是基於字符的。訓練開始時，模型不知道如何拼寫一個英文單詞，或者那個單詞甚至是一個文本單元。

*輸出的結構類似於戲劇——文本塊通常以說話者姓名開頭，所有大寫字母與數據集相似。

*如下所示，該模型在小批量文本（每個 100 個字符）上進行訓練，並且仍然能夠生成具有連貫結構的更長文本序列。

## Setup

### 導入 TensorFlow 和其他庫

In [5]:
import tensorflow as tf

import numpy as np
import os
import time

### 下載莎士比亞數據集

更改以下行以在您自己的數據上運行此代碼。

In [6]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
print(path_to_file)

1130496/1115394 [==============================] - 0s 0us/step
/root/.keras/datasets/shakespeare.txt


### 讀取數據

首先，看正文：

In [7]:
# 讀取，然後為 py2 compat 解碼。
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# 文本長度是其中的字符數
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [8]:
# 查看文本中的前 250 個字符
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [9]:
# 文件中的唯一字符
vocab = sorted(set(text))
print(vocab)
print(f'{len(vocab)} unique characters')

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65 unique characters


## 處理文本

### 向量化文本

在訓練之前，您需要將字符串轉換為數字表示。

`tf.keras.layers.StringLookup` 層可以將每個字符轉換為數字 ID。它只需要首先將文本拆分為標記。

In [10]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

現在創建 `tf.keras.layers.StringLookup` 層：

In [11]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

它將標記轉換為字符 ID：

In [12]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

由於本教程的目標是生成文本，因此反轉此表示並從中恢復人類可讀的字符串也很重要。為此，您可以使用 `tf.keras.layers.StringLookup(..., invert=True)`。

注意：這裡不是傳遞使用 `sorted(set(text))` 生成的原始詞彙表，而是使用 `tf.keras.layers.StringLookup` 層的 `get_vocabulary()` 方法，以便 `[UNK]` 標記是設置方法相同。


In [13]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

該層從 ID 的向量中恢復字符，並將它們作為字符的 `tf.RaggedTensor` 返回：

In [14]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

您可以 `tf.strings.reduce_join` 將字符重新連接成字符串。

In [15]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [16]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### 預測任務

給定一個字符或一個字符序列，最可能的下一個字符是什麼？這是您訓練模型執行的任務。模型的輸入將是一系列字符，您訓練模型以預測輸出——每個時間步的下一個字符。

由於 RNN 維持一個依賴於先前看到的元素的內部狀態，給定直到此刻計算的所有字符，下一個字符是什麼？


### 創建訓練示例和目標

接下來將文本分成示例序列。每個輸入序列都將包含文本中的“seq_length”字符。

對於每個輸入序列，對應的目標包含相同長度的文本，除了向右移動一個字符。

因此，將文本分解為 `seq_length+1` 的塊。例如，假設 `seq_length` 是 4，我們的文本是“Hello”。輸入序列是“Hell”，目標序列是“ello”。
為此，首先使用 `tf.data.Dataset.from_tensor_slices` 函數將文本向量轉換為字符索引流。

In [17]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [18]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [19]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'),end="")

First Citi

In [20]:
seq_length = 100


`batch` 方法可以讓您輕鬆地將這些單個字符轉換為所需大小的序列。

In [21]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) #分批，每批101個字

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


如果您將標記重新加入字符串，則更容易看到這是在做什麼：

In [22]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


對於訓練，您需要一個“（輸入，標籤）”對的數據集。其中“輸入”和
`label` 是序列。在每個時間步，輸入是當前字符，標籤是下一個字符。

這是一個函數，它將序列作為輸入，複製並移動它以對齊每個時間步的輸入和標籤：

In [23]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [24]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [25]:
dataset = sequences.map(split_input_target)

In [26]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### 創建訓練批次

您使用 `tf.data` 將文本拆分為可管理的序列。但在將這些數據輸入模型之前，您需要將數據打亂並打包成批次。

In [27]:
# Batch size
BATCH_SIZE = 64

#用於打亂數據集的緩衝區大小（TF 數據設計用於處理可能無限的序列，因此它不會嘗試打亂內存中的整個序列。
# 相反，它會維護一個用於打亂元素的緩衝區）。
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## 構建模型

本節將模型定義為 `keras.Model` 子類（有關詳細信息，請參閱 [通過子類化製作新層和模型](https://www.tensorflow.org/guide/keras/custom_layers_and_models)）。

該模型分為三層：

*`tf.keras.layers.Embedding`：輸入層。一個可訓練的查找表，它將每個字符 ID 映射到具有“embedding_dim”維度的向量；
*`tf.keras.layers.GRU`：一種大小為 `units=rnn_units` 的 RNN（您也可以在此處使用 LSTM 層。）
*`tf.keras.layers.Dense`：輸出層，帶有 `vocab_size` 輸出。它為詞彙表中的每個字符輸出一個 logit。這些是根據模型的每個字符的對數似然。

In [28]:
# StringLookup 層中詞彙的長度
vocab_size = len(ids_from_chars.get_vocabulary())
print(vocab_size)

# 嵌入維度
embedding_dim = 256

# RNN 單元的數量
rnn_units = 1024

66


In [29]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [30]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

對於每個字符，模型查找嵌入，以嵌入作為輸入運行 GRU 一個時間步長，並應用密集層生成預測下一個字符的對數似然的 logits：

![通過模型的數據圖](images/text_generation_training.png)

注意：對於訓練，您可以在此處使用 `keras.Sequential` 模型。要稍後生成文本，您需要管理 RNN 的內部狀態。預先包含狀態輸入和輸出選項比稍後重新排列模型架構更簡單。有關詳細信息，請參閱 [Keras RNN 指南](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse)。

## 試試模型

現在運行模型以查看其行為是否符合預期。

首先檢查輸出的形狀：

In [31]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


在上面的例子中，輸入的序列長度是“100”，但是模型可以在任何長度的輸入上運行：

In [32]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


要從模型中獲得實際預測，您需要從輸出分佈中採樣，以獲得實際的字符索引。此分佈由字符詞彙表上的 logits 定義。

注意：從這個分佈中 _sample_ 很重要，因為獲取分佈的 _argmax_ 很容易讓模型陷入循環。

嘗試批處理中的第一個示例：

In [33]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

這給了我們在每個時間步長下一個字符索引的預測：

In [34]:
sampled_indices

array([22, 57, 15,  4, 23, 12, 63, 60, 25,  3, 64, 27, 49,  2, 23, 44, 11,
       51, 54,  0, 25, 18, 49, 58, 20, 28, 49, 13, 64, 24, 64, 65, 56, 26,
       38, 29,  0, 46, 40, 35,  6, 41,  7, 63, 30, 28, 54, 27,  3, 62, 11,
       26, 31, 63, 41, 44, 40, 63, 50, 65, 29, 11, 10, 34, 34, 61,  0,  0,
       39, 40, 64, 10, 18, 42, 23,  9, 21, 37, 46, 42, 47,  5, 45, 22, 12,
       64, 54, 58, 50, 58, 56,  4,  7, 42, 39, 24, 63, 36, 59, 58])

解碼這些以查看此未經訓練的模型預測的文本：

In [35]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b" prithee, call't. For this ungentle business\nPut on thee by my lord, thou ne'er shalt see\nThy wife P"

Next Char Predictions:
 b"IrB$J;xuL!yNj Je:lo[UNK]LEjsGOj?yKyzqMYP[UNK]gaV'b,xQOoN!w:MRxbeaxkzP:3UUv[UNK][UNK]Zay3EcJ.HXgch&fI;yosksq$,cZKxWts"


## 訓練模型

此時可以將問題視為標準分類問題。給定之前的 RNN 狀態，以及這個時間步的輸入，預測下一個字符的類別

### 附加優化器和損失函數

標準的 `tf.keras.losses.sparse_categorical_crossentropy` 損失函數在這種情況下有效，因為它應用於預測的最後一個維度。

因為您的模型返回 logits，您需要設置 `from_logits` 標誌。


In [36]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [37]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1901655, shape=(), dtype=float32)


一個新初始化的模型不應該對自己太確定，輸出的 logits 應該都有相似的大小。為了確認這一點，您可以檢查平均損失的指數是否大約等於詞彙量。更高的損失意味著模型確定它的錯誤答案，並且初始化錯誤：

In [38]:
tf.exp(example_batch_mean_loss).numpy()

66.03372

使用 `tf.keras.Model.compile` 方法配置訓練過程。使用帶有默認參數和損失函數的 `tf.keras.optimizers.Adam`。

In [39]:
model.compile(optimizer='adam', loss=loss)

### 配置檢查點

使用 `tf.keras.callbacks.ModelCheckpoint` 確保在訓練期間保存檢查點：

In [40]:
# 將保存檢查點的目錄
checkpoint_dir = './training_checkpoints'
# 檢查點文件的名稱
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### 執行訓練

為了保持訓練時間合理，使用 10 個 epoch 來訓練模型。在 Colab 中，將運行時設置為 GPU 以加快訓練速度。

In [41]:
EPOCHS = 10

In [42]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 11s 48ms/step - loss: 2.7275
Epoch 2/10
172/172 [==============================] - 9s 48ms/step - loss: 1.9923
Epoch 3/10
172/172 [==============================] - 9s 49ms/step - loss: 1.7089
Epoch 4/10
172/172 [==============================] - 10s 50ms/step - loss: 1.5476
Epoch 5/10
172/172 [==============================] - 10s 49ms/step - loss: 1.4483
Epoch 6/10
172/172 [==============================] - 10s 49ms/step - loss: 1.3805
Epoch 7/10
172/172 [==============================] - 10s 50ms/step - loss: 1.3275
Epoch 8/10
172/172 [==============================] - 10s 51ms/step - loss: 1.2817
Epoch 9/10
172/172 [==============================] - 10s 51ms/step - loss: 1.2413
Epoch 10/10
172/172 [==============================] - 10s 51ms/step - loss: 1.2020


## 生成文本

使用此模型生成文本的最簡單方法是循環運行它，並在執行時跟踪模型的內部狀態。

![為了生成文本，模型的輸出被反饋到輸入](images/text_generation_sampling.png)

每次調用模型時，都會傳入一些文本和內部狀態。該模型返回對下一個字符及其新狀態的預測。將預測和狀態傳回以繼續生成文本。


下面進行單步預測：

In [43]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # 創建一個掩碼以防止生成“[UNK]”。
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # 在每個壞索引處放置一個-inf。
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # 將形狀與詞彙匹配
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # 將字符串轉換為tocken ID
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # 運行模型。
    # predict_logits.shape 是 [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # 只使用最後的預測。
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # 應用預測掩碼：防止生成“[UNK]”。
    predicted_logits = predicted_logits + self.prediction_mask

    # 對輸出 logits 進行採樣以生成令牌 ID。
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # 將令牌 ID 轉換為字符
    predicted_chars = self.chars_from_ids(predicted_ids)

    # 返回字符和模型狀態。
    return predicted_chars, states

In [44]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

在循環中運行它以生成一些文本。查看生成的文本，您會看到該模型知道何時大寫、製作段落並模仿莎士比亞式的寫作詞彙。由於訓練 epoch 的數量很少，它還沒有學會形成連貫的句子。

In [45]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Weep, do you swain, I here doth characious more:
Looking with a prestabies of his rest!
Could Bere get of slaven and woe.
Love laugh, my lords; good sir,
shall mock up the matter. Lord Hastings, say
you love: to slay behold thy news with follows
To the dogen enemy, brown hit becomes no mother.

Third Servingman:
Why, not madice!
Formar, sir, indone! Neither.

PETRUCHIO:
Dood Camillo will go Lord Stable to tempted him?

CLARENCE:
I am so bare too service.

BISHOP OF Watch!
My gracious catches victory,
Laid up't: sir, in the reep and fish, no more unvillady in the faith.

LADY ANNE:
Do plaughter'd too: sir, I am they' tender leave;
And thburrolidill is banish'd unto a warsh priviligy
Against the spear is underson my blessing:
And therefore, for a flow my mastary will determine
but neither my own desert, with heart's willings are it too;
For he more deadly York widow, tear it outry,
Which proclaim'd it is past. But so starg me no.

CORIOLANUS:
Thou call'd me, advise, I'll give me t

改善結果的最簡單方法是訓練更長時間（嘗試 `EPOCHS = 30`）。

您還可以嘗試使用不同的起始字符串，嘗試添加另一個 RNN 層以提高模型的準確性，或者調整溫度參數以生成或多或少的隨機預測。

如果您希望模型更快地生成文本，那麼您可以做的最簡單的事情就是批量生成文本。在下面的示例中，模型生成 5 個輸出的時間與上面生成 1 個輸出的時間大致相同。

In [46]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\n\nARIEL:\nUntil I am afore west begaw is for ourselves;\nFares-though forth and wedding his castle.\n\nGiFER:\nNow well, yet, let it command England's name I love your servant:\nThat better so, come, but to call.\n\nTYBAL:\nIf whether I know you this people; madam,' quak our questies beloves.\nHow, Isfire-upon Warwick, is he not to go.\n\nCAMILLO:\nWith covertural with a leaden chosen.\n\nKATHARINA:\nWell your trust, more lenctuey needless will to thy\nBonot creating Recondince to cross them.\n\nCORIOLANUS:\nWhat, I frier, that comes here!\n\nMARCIUS:\nO, sid my master?\n\nPETRUCHIO:\nSir!\n\nGREEN:\nFor whom it is your grace.\n\nBINTOPUS:\nHow! Ay, sir? not you?\n\nPOMPEY:\nI told you do it best me, not to marrieg my prote\nNo is turned of a prisoner; he's bride\nTo move, and so with weath,\nWith lap these face your sud mother, good king,\nAs mondy but after as stop, to pile upon.\n\nPETRUCHIO:\nAnd you the nurse, I will twenty good\nTo speak withal, sicked by th

## 導出生成器

這個單步模型可以很容易地[保存和恢復]（https://www.tensorflow.org/guide/saved_model），允許您在任何接受 `tf.saved_model` 的地方使用它。

In [47]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [48]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Trasser to crass the clames of the dan.
O, sir! This is hoped indeed?
Thou since we shake with Rich


## 高級：定制培訓

上面的訓練過程很簡單，但沒有給你太多的控制權。
它使用teacher-forcing 來防止錯誤的預測被反饋給模型，因此模型永遠不會學會從錯誤中恢復。

現在您已經了解瞭如何手動運行模型，接下來您將實現訓練循環。例如，如果您想實施_課程學習_以幫助穩定模型的開環輸出，這將提供一個起點。
自定義訓練循環中最重要的部分是訓練步驟函數。

使用 `tf.GradientTape` 來跟踪漸變。您可以通過閱讀 [eager execution guide](https://www.tensorflow.org/guide/eager) 了解有關此方法的更多信息。

基本程序是：

1. 執行模型併計算 `tf.GradientTape` 下的損失。
2. 計算更新並使用優化器將它們應用於模型。

In [49]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

上述 `train_step` 方法的實現遵循 [Keras 的 `train_step` 約定](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)。這是可選的，但它允許您更改訓練步驟的行為並仍然使用 keras 的 `Model.compile` 和 `Model.fit` 方法。

In [50]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [51]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [52]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 12s 52ms/step - loss: 2.7231


或者，如果您需要更多控制，您可以編寫自己的完整自定義訓練循環：

In [53]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1632
Epoch 1 Batch 50 Loss 2.0944
Epoch 1 Batch 100 Loss 1.9545
Epoch 1 Batch 150 Loss 1.8317

Epoch 1 Loss: 1.9877
Time taken for 1 epoch 10.45 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8241
Epoch 2 Batch 50 Loss 1.7352
Epoch 2 Batch 100 Loss 1.6738
Epoch 2 Batch 150 Loss 1.6375

Epoch 2 Loss: 1.7069
Time taken for 1 epoch 9.86 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5902
Epoch 3 Batch 50 Loss 1.5801
Epoch 3 Batch 100 Loss 1.5222
Epoch 3 Batch 150 Loss 1.5138

Epoch 3 Loss: 1.5455
Time taken for 1 epoch 9.91 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.5004
Epoch 4 Batch 50 Loss 1.4161
Epoch 4 Batch 100 Loss 1.4511
Epoch 4 Batch 150 Loss 1.4294

Epoch 4 Loss: 1.4465
Time taken for 1 epoch 9.97 sec
________________________________________________________________________